In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_react_agent, AgentExecutor
import requests
from langchain import hub

In [4]:
search_tool = DuckDuckGoSearchRun()

search_tool.invoke("What is gen AI")

d:\Gen AI\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


"4 days ago · What is General AI? Artificial General Intelligence (AGI) or General AI represents a theoretical form of artificial intelligence (AI) that can solve any task using human-like cognitive … Jan 31, 2025 · Please use this conversation thread to discuss Gen AI in Design held on Jan 31, 2025, 3PM, Pacific Time. Feel free to ask questions about the place, date and time, agenda of … Meetup Details Hi IxDF Geneva, let’s meet online and discuss UX! In our video call, let’s get to know each other and discuss the latest digital product designs we like. Beyond this, what are … 2 days ago · AI-generated art is artwork created with the assistance of artificial intelligence (AI). This process typically involves algorithms that can learn from data inputs and generate new, … Online, self-paced UX Courses created by design experts. Join over 189, 985 students in the world' s largest design school and gain recognized certificates."

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
prompt = hub.pull("hwchase17/react") # pulls the standard ReAct agent prompt
# it is like a github for prompts

print(prompt.template)

d:\Gen AI\LangChain\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [11]:
agent = create_react_agent(
    llm = llm,
    tools = [search_tool],
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = [search_tool],
    verbose = True
)

In [18]:
response = agent_executor.invoke({"input": "What is the score of Shubman Gill in yesterdays test match between England and India"})



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: Shubman Gill score yesterday England India test match

d:\Gen AI\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


5 days ago · England face India in the second Test at Edgbaston - follow live scores, radio commentary, video highlights and updates. 1 day ago · Team India created history at Edgbaston as they registered their first-ever Test victory at the iconic venue, defeating England by a massive 336-run margin in the second Test of the … 18 hours ago · Shubman Gill’s record-breaking innings powered India to their first Test win in Birmingham. The victory dealt a heavy blow to England’s Bazball era, raising questions about … 1 day ago · India vs England, 2nd Test Match Report: Shubman Gill tallied 430 runs and Akash Deep, Mohammed Siraj both picked up five-wicket hauls as the visitors dismissed Ben Stokes and Co … 1 day ago · A record breaking Test match from Shubman Gill and some quality fast bowling by Akash Deep and Mohammed Siraj ensured India clinched the 2nd test in Birmingham. Set a target …Final Answer: Shubman Gill scored 430 runs in yesterday's test match between England and India.

> F

In [19]:
print(response)

{'input': 'What is the score of Shubman Gill in yesterdays test match between England and India', 'output': "Shubman Gill scored 430 runs in yesterday's test match between England and India."}


### Search tool + Weather API

In [20]:
@tool
def get_weather_data(city: str) -> str:
    """This function fetches the current weather data for a given city"""
    url = f"https://api.weatherapi.com/v1/current.json?key=05ae5911a4864a5fbca120949250707&q={city}" # weatherapi.com
    response = requests.get(url)
    return response.json()

In [21]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [22]:
response = agent_executor.invoke({"input": "What is the capital of Telangana and it's current weather condition"})



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: capital of Telangana

d:\Gen AI\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


6 days ago · Telangana is a state of south-central India. The area of what is now Telangana constituted the north … 4 days ago · Telangana – Hyderabad serves as the capital of India’s youngest state, formed in 2014. Kerala – … Aug 4, 2024 · The Hyderabad is the capital of Telangana State. South of North and North of South,’ Telangana State … Feb 7, 2025 · The city of Hyderabad officially ceases to be the joint capital of Telangana and Andhra Pradesh on 2 June 2024. On 2 June 2014, Hyderabad was made the joint capital of Andhra Pradesh and the newly created state of Telangana for ten years. Telangana was created as the 29th state of India by bifurcating Andhra Pradesh in 2014. Status of Hyderabad 2 days ago · Hyderabad district has the capital city of Telangana, also called Hyderabad, which is the head office of the district. The …Action: get_weather_data
Action Input: Hyderabad{'location': {'name': 'Hyderabad', 'region': 'Telangana', 'country': 'India', 'lat': 17.3753, 'lon': 78.4744, 

In [23]:
response

{'input': "What is the capital of Telangana and it's current weather condition",
 'output': 'The capital of Telangana is Hyderabad. The current weather condition in Hyderabad is Mist with a temperature of 29.3°C.'}